In [ ]:
from lyrics_db import conn

In [ ]:
stmt = """
SELECT a.album_name, s.song_name, s.embedding
    FROM albums AS a
    INNER JOIN songs AS s on s.album_id = a.id
    WHERE a.album_name in ('Blood on the Tracks', 'Bob Dylan', 'Björk', 'The Chirping Crickets')
    ORDER BY a.album_name, s.song_number
"""

In [ ]:
conn.rollback()
cur = conn.cursor()
cur.execute(stmt)
songs = cur.fetchall()
cur.close()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

def plot_embeddings(embedding_list, album_names, songs_per_album):
    c = 0
    colors = ['red', 'green', 'blue', 'black', 'orange', 'cyan', 'indigo', 'darkblue', 'darkgreen', 'fuchsia']
    color_indices = []
    matrix = None
    for embedding in embedding_list:
        if matrix is None:
            matrix = np.array(embedding)
        else:
            matrix = np.vstack((matrix, np.array(embedding)))
    for songs in songs_per_album:
        color_indices.extend(pd.Series([c for x in range(songs)]))
        c = (c + 1) % len(colors)
    tsne = TSNE(
        n_components=2, perplexity=4, random_state=42, init="random", learning_rate=10
    )
    vis_dims = tsne.fit_transform(matrix)
    x = [x for x,y in vis_dims]
    y = [y for x,y in vis_dims]
    
    fig, ax = plt.subplots()
    colormap = matplotlib.colors.ListedColormap(colors)
    scatter = ax.scatter(x, y, c=color_indices, cmap=colormap, alpha=0.8)
    plots, _labels = scatter.legend_elements()
    legend = ax.legend(plots, album_names, title="Albums")
    ax.add_artist(legend)
    plt.show()    

In [ ]:
embedding_list = []
album_names = []
songs_per_album = []
for s in songs:
    embedding_list.append(eval(s[2]))
    if s[0] not in album_names:
        album_names.append(s[0])
        songs_per_album.append(0)
    songs_per_album[-1] += 1

In [ ]:
plot_embeddings(embedding_list, album_names, songs_per_album)